In [14]:
from pynq import Xlnk, DefaultIP, DefaultHierarchy, Overlay
import pynq.lib.dma
import numpy

d = None

class HSDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)
        
        
    bindto = ['xilinx.com:hls:hailstone:1.0']
        
    @property
    def buf_len(self): 
        return self.read(0x10)
    
    @buf_len.setter
    def buf_len(self, value): 
        self.write(0x10, value)


class HSStreamDriver(DefaultHierarchy):
    def __init__(self, description):
        super().__init__(description=description)
    
    def hailstone(self, stream):
        b_len = len(stream)
        self.hailstone.buf_len = b_len
        with xlnk.cma_array(shape=(b_len,), \
                            dtype=numpy.uint32) as in_buffer,\
        xlnk.cma_array(shape=(b_len,), \
                       dtype=numpy.uint32) as out_buffer:
    
            for i, v in enumerate(stream):
                in_buffer[i] = v
            
            self.hailstone_dma.sendchannel.transfer(in_buffer)
            self.hailstone_dma.recvchannel.transfer(out_buffer)
            self.hailstone_dma.sendchannel.wait()
            self.hailstone_dma.recvchannel.wait()
            result = out_buffer.copy()            
        return result
    
    
    @staticmethod
    def checkhierarchy(description):
        if 'hailstone_dma' in description['ip']\
        and 'hailstone'in description['ip']:
            return True
        return False
       
    
hs_ol = Overlay('/home/xilinx/jupyter_notebooks/dmatest.bit')
help(hs_ol)

Help on Overlay in module pynq.overlay object:

class Overlay(pynq.pl.Bitstream)
 |  This class keeps track of a single bitstream's state and contents.
 |  
 |  The overlay class holds the state of the bitstream and enables run-time
 |  protection of bindlings.
 |  
 |  Our definition of overlay is: "post-bitstream configurable design".
 |  Hence, this class must expose configurability through content discovery
 |  and runtime protection.
 |  
 |  The overlay class exposes the IP and hierarchies as attributes in the
 |  overlay. If no other drivers are available the `DefaultIP` is constructed
 |  for IP cores at top level and `DefaultHierarchy` for any hierarchies that
 |  contain addressable IP. Custom drivers can be bound to IP and hierarchies
 |  by subclassing `DefaultIP` and `DefaultHierarchy`. See the help entries
 |  for those class for more details.
 |  
 |  This class stores four dictionaries: IP, GPIO, interrupt controller
 |  and interrupt pin dictionaries.
 |  
 |  Each ent

In [17]:
s = 2
e = 3

stream = [x for x in range(s,e)]

buf_len = len(stream)
hs_ol.hailstone.buf_len = buf_len
xlnk = Xlnk()

in_buffer = xlnk.cma_array(shape=(buf_len,), dtype = numpy.uint32)
for i,v in enumerate(stream):
    in_buffer[i] = v
    print(in_buffer[i])

out_buffer = xlnk.cma_array(shape=(buf_len,), dtype = numpy.uint32)



hs_ol.hailstone_dma.sendchannel.transfer(in_buffer)
hs_ol.hailstone_dma.recvchannel.transfer(out_buffer)

hs_ol.hailstone_dma.sendchannel.wait()
hs_ol.hailstone_dma.recvchannel.wait()
out_buffer

2


KeyboardInterrupt: 

In [15]:
hs_ol?


In [18]:
checkhierarchy(hs_ol?)

SyntaxError: invalid syntax (<ipython-input-18-c1150c9df9b1>, line 1)